# Making old trained models compatible with lightning

TC: https://wandb.ai/gnn_tracking/gnn_tracking/runs/04b2e3ce/
GC: https://wandb.ai/gnn_tracking/gnn_tracking_gc/runs/7dce6aff
EC: https://wandb.ai/gnn_tracking/gnn_tracking_ec/runs/ddff435e

In [7]:
%load_ext autoreload
%autoreload 2

from gnn_tracking.models.graph_construction import GraphConstructionFCNN
from gnn_tracking.utils.loading import TrackingDataModule
import torch
from gnn_tracking.training.ml import MLModule
from functools import partial
from gnn_tracking.metrics.losses import GraphConstructionHingeEmbeddingLoss, \
    HaughtyFocalLoss
from pytorch_lightning import Trainer
from gnn_tracking.training.ec import ECModule
from gnn_tracking.models.edge_classifier import ec_from_chkpt
import pprint
from gnn_tracking.models.graph_construction import MLGraphConstruction
from gnn_tracking.training.tc import TCModule

In [41]:
dm = TrackingDataModule(
    train=dict(
        dirs=[
            "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/"
        ],
        # If you run into memory issues, reduce this
    ),
    val=dict(
        dirs=[
            "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/"
        ],
        stop=5
    ),
)

In [9]:
dm.setup(stage="fit")

[17:24:56] INFO: DataLoader will load 900 graphs (out of 900 available).
[17:24:56] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/data21000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/data21999_s0.pt
[17:24:56] INFO: DataLoader will load 1 graphs (out of 1000 available).
[17:24:56] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/data29000_s0.pt


In [10]:
from pathlib import Path

model_exchange_path = Path(
    "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/model_exchange"
)

## ML

In [5]:
gc = GraphConstructionFCNN(in_dim=14, out_dim=8, depth=6, hidden_dim=512, beta=0.4)


In [10]:

ckpt = torch.load("/home/kl5675/ray_results/gc-hinge-sq-sq-cells/GCTrainable_7dce6aff_24_val_batch_size=1,adam_amsgrad=False,adam_beta1=0.9000,adam_beta2=0.9990,adam_eps=0.0000,adam_weight_decay=_2023-06-08_13-32-02/checkpoint_000009/checkpoint.pt", map_location="cpu")
state_dct = ckpt["model_state_dict"]
state_dct = {f"_{key}": value for key, value in state_dct.items()}
gc.load_state_dict(state_dct)

<All keys matched successfully>

In [11]:

lmodel = MLModule(
    model=gc,
    loss_fct=GraphConstructionHingeEmbeddingLoss(max_num_neighbors=256, r_emb=1, attr_pt_thld=0.9),
    lw_repulsive=0.001953029788887701,  # loss weight, see below
    optimizer=partial(torch.optim.Adam, lr=1e-3),
)

In [12]:

trainer = Trainer(accelerator="cpu", max_epochs=0)

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3 ...
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [13]:
trainer.fit(lmodel, dm)

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3 ...
  rank_zero_warn(
[16:29:05] INFO: DataLoader will load 900 graphs (out of 900 available).
[16:29:05] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/data21000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/data21999_s0.pt
[16:29:05] INFO: DataLoader will load 5 graphs (out of 1000 available).
[16:29:05] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/da

Sanity Checking: 0it [00:00, ?it/s]

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=0` reached.


In [14]:
trainer.save_checkpoint(model_exchange_path / "gc-7dce6aff.ckpt")

In [16]:
lmodel.hparams

"loss_fct":     {'class_path': 'gnn_tracking.metrics.losses.GraphConstructionHingeEmbeddingLoss', 'init_args': {'r_emb': 1, 'max_num_neighbors': 256, 'attr_pt_thld': 0.9, 'p_attr': 1, 'p_rep': 1}}
"lw_repulsive": 0.001953029788887701
"model":        {'class_path': 'gnn_tracking.models.graph_construction.GraphConstructionFCNN', 'init_args': {'in_dim': 14, 'hidden_dim': 512, 'out_dim': 8, 'depth': 6, 'beta': 0.4}}
"preproc":      None

## Trying to restore

In [11]:
lmodel = MLGraphConstruction.from_chkpt(
    ml_chkpt_path=model_exchange_path / "gc" / "gc-7dce6aff.ckpt",
    ml_freeze=False,
)

[17:24:59] DEBUG: Getting class MLModule from module gnn_tracking.training.ml
[17:24:59] DEBUG: Loading checkpoint /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/model_exchange/gc/gc-7dce6aff.ckpt
[17:24:59] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[17:24:59] DEBUG: Getting class GraphConstructionHingeEmbeddingLoss from module gnn_tracking.metrics.losses
[17:24:59] DEBUG: Checkpoint loaded. Model ready to go.


In [12]:
out = lmodel(dm.datasets["val"][0])

In [19]:
out

Data(x=[59357, 14], edge_index=[2, 7837097], edge_attr=[7837097, 28], y=[7837097], pt=[59357], particle_id=[59357], sector=[59357], reconstructable=[59357])

## EC

In [6]:
from gnn_tracking.models.edge_classifier import ECForGraphTCN

model = ECForGraphTCN(
    node_indim=14+8,
    edge_indim=(14+8)*2,
    interaction_node_dim=128,
    interaction_edge_dim=128,
    hidden_dim=128,
    L_ec=3,
    alpha=0.35,
    use_intermediate_edge_embeddings=False,
    use_node_embedding=True,
)


In [7]:
ckpt = torch.load("/home/kl5675/ray_results/ds-ef-7dce6aff/GCWithECTrainable_ddff435e_1_val_batch_size=1,adam_amsgrad=False,adam_beta1=0.9000,adam_beta2=0.9990,adam_eps=0.0000,adam_weight_d_2023-06-10_16-10-21/checkpoint_000083/checkpoint.pt", map_location="cpu")
state_dct = ckpt["model_state_dict"]
model.load_state_dict(state_dct)

<All keys matched successfully>

In [8]:


lmodel = ECModule(
    model=model,
    loss_fct=HaughtyFocalLoss(alpha=0.5, gamma=2),
    preproc=MLGraphConstruction.from_chkpt(
        ml_chkpt_path=model_exchange_path / "gc" / "gc-7dce6aff.ckpt",
        use_embedding_features=True,
    )
)

[16:33:24] DEBUG: Getting class MLModule from module gnn_tracking.training.ml
[16:33:24] DEBUG: Loading checkpoint /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/model_exchange/gc/gc-7dce6aff.ckpt
[16:33:24] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[16:33:24] DEBUG: Getting class GraphConstructionHingeEmbeddingLoss from module gnn_tracking.metrics.losses
[16:33:24] DEBUG: Checkpoint loaded. Model ready to go.


In [9]:
trainer = Trainer(accelerator="cpu", max_epochs=0, num_sanity_val_steps=0)

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3 ...
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [10]:
trainer.fit(lmodel, dm)

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3 ...
  rank_zero_warn(
[16:33:26] INFO: DataLoader will load 900 graphs (out of 900 available).
[16:33:26] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/data21000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/data21999_s0.pt
[16:33:26] INFO: DataLoader will load 5 graphs (out of 1000 available).
[16:33:26] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/da

In [11]:
trainer.save_checkpoint(model_exchange_path / "ec" / "ec-ddff435e.ckpt")

### Try restore

In [15]:
MLGraphConstruction.from_chkpt(
    ml_chkpt_path=model_exchange_path / "gc" / "gc-7dce6aff.ckpt",
    ec_chkpt_path=model_exchange_path / "ec" / "ec-ddff435e.ckpt",
    use_embedding_features=True,
    ec_threshold=0.5,
)

[17:25:28] DEBUG: Getting class MLModule from module gnn_tracking.training.ml
[17:25:28] DEBUG: Loading checkpoint /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/model_exchange/gc/gc-7dce6aff.ckpt
[17:25:28] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[17:25:28] DEBUG: Getting class GraphConstructionHingeEmbeddingLoss from module gnn_tracking.metrics.losses
[17:25:28] DEBUG: Checkpoint loaded. Model ready to go.
[17:25:28] DEBUG: Getting class ECModule from module gnn_tracking.training.ec
[17:25:28] DEBUG: Loading checkpoint /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/model_exchange/ec/ec-ddff435e.ckpt
[17:25:28] DEBUG: Getting class ECForGraphTCN from module gnn_tracking.models.edge_classifier
[17:25:28] DEBUG: Getting class MLGraphConstruction from module gnn_tracking.models.graph_construction
[17:25:28] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[17:25:28] DEBUG: G

MLGraphConstruction(
  (_ml): GraphConstructionFCNN(
    (_encoder): Linear(in_features=14, out_features=512, bias=False)
    (_decoder): Linear(in_features=512, out_features=8, bias=False)
    (_layers): ModuleList(
      (0-4): 5 x Linear(in_features=512, out_features=512, bias=False)
    )
  )
  (_ef): ECForGraphTCN(
    (relu): ReLU()
    (ec_node_encoder): MLP(
      (layers): ModuleList(
        (0): Linear(in_features=22, out_features=128, bias=False)
        (1): ReLU()
        (2): Linear(in_features=128, out_features=128, bias=False)
      )
    )
    (ec_edge_encoder): MLP(
      (layers): ModuleList(
        (0): Linear(in_features=44, out_features=128, bias=False)
        (1): ReLU()
        (2): Linear(in_features=128, out_features=128, bias=False)
      )
    )
    (ec_resin): ResIN(
      (network): Skip1ResidualNetwork(
        (layers): ModuleList(
          (0-2): 3 x InteractionNetwork()
        )
      )
    )
    (W): MLP(
      (layers): ModuleList(
        (0): 

In [13]:

ec_restored = ec_from_chkpt(model_exchange_path / "ec" / "ec-ddff435e.ckpt")

[16:34:44] DEBUG: Getting class ECModule from module gnn_tracking.training.ec
[16:34:44] DEBUG: Loading checkpoint /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/model_exchange/ec/ec-ddff435e.ckpt
[16:34:44] DEBUG: Getting class ECForGraphTCN from module gnn_tracking.models.edge_classifier
[16:34:44] DEBUG: Getting class MLGraphConstruction from module gnn_tracking.models.graph_construction
[16:34:44] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[16:34:44] DEBUG: Getting class HaughtyFocalLoss from module gnn_tracking.metrics.losses
[16:34:44] DEBUG: Checkpoint loaded. Model ready to go.


## TC

In [24]:
from pytorch_lightning.core.mixins import HyperparametersMixin
from gnn_tracking.models.track_condensation_networks import GraphTCN, \
    PreTrainedECGraphTCN

# class MockEC():
#     latent_dim=(128, 128)

model = PreTrainedECGraphTCN(
    ec=ec_from_chkpt(model_exchange_path / "ec" / "ec-ddff435e.ckpt"),
    node_indim=22,
    edge_indim=44,
    h_dim=192,
    e_dim=192,
    h_outdim=12,
    hidden_dim=192,
    L_hc=3,
    feed_edge_weights=True,
    ec_threshold=0.2,
    mask_orphan_nodes=False,
    use_ec_embeddings_for_hc=True,

)

[17:32:26] DEBUG: Getting class ECModule from module gnn_tracking.training.ec
[17:32:26] DEBUG: Loading checkpoint /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/model_exchange/ec/ec-ddff435e.ckpt
[17:32:26] DEBUG: Getting class ECForGraphTCN from module gnn_tracking.models.edge_classifier
[17:32:26] DEBUG: Getting class MLGraphConstruction from module gnn_tracking.models.graph_construction
[17:32:26] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[17:32:26] DEBUG: Getting class HaughtyFocalLoss from module gnn_tracking.metrics.losses
[17:32:26] DEBUG: Checkpoint loaded. Model ready to go.
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:196: UserWarning: Attribute 'ec' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['ec'])`.
  rank_zero_warn(
/scratch/gpfs/kl5675/micromamba/

In [25]:
model.hparams

"L_hc":                     3
"alpha_hc":                 0.5
"e_dim":                    192
"ec":                       {'class_path': 'gnn_tracking.models.edge_classifier.ECForGraphTCN', 'init_args': {'node_indim': 22, 'edge_indim': 44, 'interaction_node_dim': 128, 'interaction_edge_dim': 128, 'hidden_dim': 128, 'L_ec': 3, 'alpha': 0.35, 'residual_type': 'skip1', 'use_intermediate_edge_embeddings': False, 'use_node_embedding': True, 'residual_kwargs': None}}
"ec_threshold":             0.2
"edge_indim":               44
"feed_edge_weights":        True
"h_dim":                    192
"h_outdim":                 12
"hidden_dim":               192
"mask_orphan_nodes":        False
"node_indim":               22
"use_ec_embeddings_for_hc": True

In [26]:
ckpt = torch.load("/home/kl5675/ray_results/613788bb-continued/TCNFromGCTrainable_04b2e3ce_2_val_batch_size=1,adam_amsgrad=False,adam_beta1=0.9000,adam_beta2=0.9990,adam_eps=0.0000,adam_weight__2023-06-14_23-29-18/checkpoint_000018/checkpoint.pt", map_location="cpu")
state_dct = {k: v for k, v in ckpt["model_state_dict"].items() if not k.startswith("_gtcn.ec")}
model.load_state_dict(state_dct, strict=False)

_IncompatibleKeys(missing_keys=['_gtcn.ec.ec_node_encoder.layers.0.weight', '_gtcn.ec.ec_node_encoder.layers.2.weight', '_gtcn.ec.ec_edge_encoder.layers.0.weight', '_gtcn.ec.ec_edge_encoder.layers.2.weight', '_gtcn.ec.ec_resin.network.layers.0.relational_model.layers.0.weight', '_gtcn.ec.ec_resin.network.layers.0.relational_model.layers.0.bias', '_gtcn.ec.ec_resin.network.layers.0.relational_model.layers.2.weight', '_gtcn.ec.ec_resin.network.layers.0.relational_model.layers.2.bias', '_gtcn.ec.ec_resin.network.layers.0.relational_model.layers.4.weight', '_gtcn.ec.ec_resin.network.layers.0.relational_model.layers.4.bias', '_gtcn.ec.ec_resin.network.layers.0.object_model.layers.0.weight', '_gtcn.ec.ec_resin.network.layers.0.object_model.layers.0.bias', '_gtcn.ec.ec_resin.network.layers.0.object_model.layers.2.weight', '_gtcn.ec.ec_resin.network.layers.0.object_model.layers.2.bias', '_gtcn.ec.ec_resin.network.layers.0.object_model.layers.4.weight', '_gtcn.ec.ec_resin.network.layers.0.objec

In [51]:

from gnn_tracking.postprocessing.dbscanscanner import DBSCANHyperParamScanner

lmodel = TCModule(
    model=model,
    lw_repulsive=0.743380428762342,
    lw_background=0.0041,
    preproc=MLGraphConstruction.from_chkpt(
        ml_chkpt_path=model_exchange_path / "gc" / "gc-7dce6aff.ckpt",
        use_embedding_features=True,
        max_num_neighbors=64,
        max_radius=0.8,
    ),
    cluster_scanner=DBSCANHyperParamScanner(n_trials=20, n_jobs=6),
)

[17:38:35] DEBUG: Getting class MLModule from module gnn_tracking.training.ml
[17:38:35] DEBUG: Loading checkpoint /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/model_exchange/gc/gc-7dce6aff.ckpt
[17:38:35] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[17:38:35] DEBUG: Getting class GraphConstructionHingeEmbeddingLoss from module gnn_tracking.metrics.losses
[17:38:35] DEBUG: Checkpoint loaded. Model ready to go.


In [52]:
pprint.pprint(dict(lmodel.hparams))

{'background_loss': {'class_path': 'gnn_tracking.metrics.losses.BackgroundLoss',
                     'init_args': {'sb': 0.1}},
 'cluster_scanner': {'class_path': 'gnn_tracking.postprocessing.dbscanscanner.DBSCANHyperParamScanner',
                     'init_args': {'eps_range': (1e-05, 1.0),
                                   'guide': 'trk.double_majority_pt0.9',
                                   'min_samples_range': (1, 1),
                                   'n_jobs': 6,
                                   'n_trials': 20}},
 'lw_background': 0.0041,
 'lw_repulsive': 0.743380428762342,
 'model': {'class_path': 'gnn_tracking.models.track_condensation_networks.PreTrainedECGraphTCN',
           'init_args': {'L_hc': 3,
                         'alpha_hc': 0.5,
                         'e_dim': 192,
                         'ec': {'class_path': 'gnn_tracking.models.edge_classifier.ECForGraphTCN',
                                'init_args': {'L_ec': 3,
                                   

In [54]:
trainer = Trainer(accelerator="cpu", max_steps=0, num_sanity_val_steps=0)

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3 ...
  rank_zero_warn(
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [55]:
trainer.fit(lmodel, dm)

[17:38:46] INFO: DataLoader will load 900 graphs (out of 900 available).
[17:38:46] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/data21000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/data21999_s0.pt
[17:38:46] INFO: DataLoader will load 5 graphs (out of 1000 available).
[17:38:46] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/data29004_s0.pt

  | Name            | Type                 | Params
---------------------------------------------------------
0 | model           | PreTrainedECGraphTCN | 1.8 M 
1 | preproc         | MLGraphConstruction  | 1.3 M 
2 | potential_loss  | PotentialLoss        | 0     
3 | background_loss | BackgroundLoss       | 0     
---------------------------------------------------------
1.3 M 

In [56]:
trainer.save_checkpoint(model_exchange_path / "tc" / "tc-04b2e3ce.ckpt")

## Try restore

In [57]:
lmodel = TCModule.load_from_checkpoint(
    model_exchange_path / "tc" / "tc-04b2e3ce.ckpt",
)

[17:38:48] DEBUG: Getting class PreTrainedECGraphTCN from module gnn_tracking.models.track_condensation_networks
[17:38:48] DEBUG: Getting class ECForGraphTCN from module gnn_tracking.models.edge_classifier
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:196: UserWarning: Attribute 'ec' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['ec'])`.
  rank_zero_warn(
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:196: UserWarning: Attribute 'hc_in' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['hc_in'])`.
  rank_zero_warn(
[17:38:48] DEBUG: Getting class MLGraphConstruction from module gnn_tracking.models.graph_construction
[17:38:48] DEBUG: Getting class GraphConstructionFCN

In [62]:
from rich.pretty import pprint

pprint(dict(lmodel.hparams), )

{
│   'model': {
│   │   'class_path': 'gnn_tracking.models.track_condensation_networks.PreTrainedECGraphTCN',
│   │   'init_args': {
│   │   │   'node_indim': 22,
│   │   │   'edge_indim': 44,
│   │   │   'h_dim': 192,
│   │   │   'e_dim': 192,
│   │   │   'h_outdim': 12,
│   │   │   'hidden_dim': 192,
│   │   │   'L_hc': 3,
│   │   │   'alpha_hc': 0.5,
│   │   │   'feed_edge_weights': True,
│   │   │   'ec_threshold': 0.2,
│   │   │   'mask_orphan_nodes': False,
│   │   │   'use_ec_embeddings_for_hc': True,
│   │   │   'ec': {
│   │   │   │   'class_path': 'gnn_tracking.models.edge_classifier.ECForGraphTCN',
│   │   │   │   'init_args': {
│   │   │   │   │   'node_indim': 22,
│   │   │   │   │   'edge_indim': 44,
│   │   │   │   │   'interaction_node_dim': 128,
│   │   │   │   │   'interaction_edge_dim': 128,
│   │   │   │   │   'hidden_dim': 128,
│   │   │   │   │   'L_ec': 3,
│   │   │   │   │   'alpha': 0.35,
│   │   │   │   │   'residual_type': 'skip1',
│   │   │   │   │   'use_intermediate_edge_embeddings': False,
│   │   │   │   │   'use_node_embedding': True,
│   │   │   │   │   'residual_kwargs': None
│   │   │   │   }
│   │   │   }
│   │   }
│   },
│   'preproc': {
│   │   'class_path': 'gnn_tracking.models.graph_construction.MLGraphConstruction',
│   │   'init_args': {
│   │   │   'max_radius': 0.8,
│   │   │   'max_num_neighbors': 64,
│   │   │   'use_embedding_features': True,
│   │   │   'ratio_of_false': None,
│   │   │   'build_edge_features': True,
│   │   │   'ec_threshold': None,
│   │   │   'ml_freeze': True,
│   │   │   'ec_freeze': True,
│   │   │   'ml': {
│   │   │   │   'class_path': 'gnn_tracking.models.graph_construction.GraphConstructionFCNN',
│   │   │   │   'init_args': {'in_dim': 14, 'hidden_dim': 512, 'out_dim': 8, 'depth': 6, 'beta': 0.4}
│   │   │   },
│   │   │   'ec': None
│   │   }
│   },
│   'lw_repulsive': 0.743380428762342,
│   'lw_background': 0.0041,
│   'potential_loss': {
│   │   'class_path': 'gnn_tracking.metrics.losses.PotentialLoss',
│   │   'init_args': {'q_min': 0.01, 'radius_threshold': 1.0, 'attr_pt_thld': 0.9}
│   },
│   'background_loss': {'class_path': 'gnn_tracking.metrics.losses.BackgroundLoss', 'init_args': {'sb': 0.1}},
│   'cluster_scanner': {
│   │   'class_path': 'gnn_tracking.postprocessing.dbscanscanner.DBSCANHyperParamScanner',
│   │   'init_args': {
│   │   │   'eps_range': (1e-05, 1.0),
│   │   │   'min_samples_range': (1, 1),
│   │   │   'n_jobs': 6,
│   │   │   'guide': 'trk.double_majority_pt0.9',
│   │   │   'n_trials': 20
│   │   }
│   }
}

In [59]:
trainer = Trainer(accelerator="gpu", max_steps=100, num_sanity_val_steps=0)

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3 ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [40]:
trainer.fit(lmodel, dm)

You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
[17:33:22] INFO: DataLoader will load 900 graphs (out of 900 available).
[17:33:22] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/data21000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_1/data21999_s0.pt
[17:33:22] INFO: DataLoader will load 1 graphs (out of 1000 available).
[17:33:22] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/data29000_s0.pt
LOCAL_RANK: 0 - CUDA_

Training: 0it [00:00, ?it/s]

[17:33:29] WARNING: WARNING: ran out of memory (OOM), skipping batch. If this happens frequently, decrease the batch size. Will abort if we get 10 consecutive OOM errors.
[17:33:34] WARNING: WARNING: ran out of memory (OOM), skipping batch. If this happens frequently, decrease the batch size. Will abort if we get 10 consecutive OOM errors.
[17:33:41] WARNING: WARNING: ran out of memory (OOM), skipping batch. If this happens frequently, decrease the batch size. Will abort if we get 10 consecutive OOM errors.
[17:33:47] WARNING: WARNING: ran out of memory (OOM), skipping batch. If this happens frequently, decrease the batch size. Will abort if we get 10 consecutive OOM errors.
[17:33:51] WARNING: WARNING: ran out of memory (OOM), skipping batch. If this happens frequently, decrease the batch size. Will abort if we get 10 consecutive OOM errors.
[17:34:04] WARNING: WARNING: ran out of memory (OOM), skipping batch. If this happens frequently, decrease the batch size. Will abort if we get 1

In [60]:
dm.setup("validate")

[17:39:05] INFO: DataLoader will load 5 graphs (out of 1000 available).
[17:39:05] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/data29004_s0.pt


In [61]:
trainer.validate(lmodel, dm, ckpt_path="last")

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:189: UserWarning: .validate(ckpt_path="last") is set, but there is no last checkpoint available. No checkpoint will be loaded.
  rank_zero_warn(
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
[17:39:05] INFO: DataLoader will load 5 graphs (out of 1000 available).
[17:39:05] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part_9/data29004_s0.pt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/scratch/gpf

Validation: 0it [00:00, ?it/s]

[17:39:13 ClusterHP] DEBUG: Starting from params: {}
[17:39:13 ClusterHP] INFO: Starting hyperparameter scan for clustering
[17:39:18 ClusterHP] DEBUG: Evaluated {'eps': 0.2049804544112404, 'min_samples': 1}: 0.7014782803132903
[17:39:23 ClusterHP] DEBUG: Evaluated {'eps': 0.15190959634811918, 'min_samples': 1}: 0.49863744176644553
[17:39:28 ClusterHP] DEBUG: Evaluated {'eps': 0.519082719358773, 'min_samples': 1}: 0.8585183619846294
[17:39:31 ClusterHP] DEBUG: Evaluated {'eps': 0.5809714917578661, 'min_samples': 1}: 0.8172594076081499
[17:39:33 ClusterHP] DEBUG: Evaluated {'eps': 0.1971795917526922, 'min_samples': 1}: 0.6796676483704633
[17:39:33 ClusterHP] DEBUG: Evaluated {'eps': 0.4084848020408636, 'min_samples': 1}: 0.8916953972180843
[17:39:34 ClusterHP] DEBUG: Evaluated {'eps': 0.015105911698334206, 'min_samples': 1}: 0.0
[17:39:42 ClusterHP] DEBUG: Evaluated {'eps': 0.32878847832290337, 'min_samples': 1}: 0.8744153624910472
[17:39:43 ClusterHP] DEBUG: Evaluated {'eps': 0.7698647

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃            Validate metric             ┃              DataLoader 0              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│             adjusted_rand              │          0.03215637058019638           │
│           adjusted_rand_std            │         0.0037478802260011435          │
│         adjusted_rand_std_std          │                  nan                   │
│               attractive               │         1.3140039300196804e-05         │
│             attractive_std             │         2.9810652790729364e-07         │
│          attractive_weighted           │         1.3140039300196804e-05         │
│        attractive_weighted_std         │         2.9810652790729364e-07         │
│               background               │                  1.0                   │
│             background_std             │         1.332800358966324e-08          │
│          background_weighted           │          0.00409999955445528           │
│        background_weighted_std         │         1.0412502804424406e-10         │
│            best_dbscan_eps             │           0.4084848165512085           │
│          best_dbscan_eps_std           │                  nan                   │
│        best_dbscan_min_samples         │                  1.0                   │
│      best_dbscan_min_samples_std       │                  nan                   │
│              completeness              │           0.8834384679794312           │
│            completeness_std            │          0.003333359956741333          │
│          completeness_std_std          │                  nan                   │
│            fowlkes_mallows             │          0.03728548809885979           │
│          fowlkes_mallows_std           │         0.0034563906956464052          │
│        fowlkes_mallows_std_std         │                  nan                   │
│              homogeneity               │           0.9336580634117126           │
│            homogeneity_std             │          0.006325864233076572          │
│          homogeneity_std_std           │                  nan                   │
│               repulsive                │         2.991704059240874e-05          │
│             repulsive_std              │         1.1564778787942487e-06         │
│           repulsive_weighted           │         2.223974297521636e-05          │
│         repulsive_weighted_std         │         8.597029932388978e-07          │
│                 total                  │          0.004135379567742348          │
│               total_std                │         1.1014168421752402e-06         │
│          trk.double_majority           │           0.5153970718383789           │
│       trk.double_majority_pt0.5        │           0.7771095037460327           │
│     trk.double_majority_pt0.5_std      │          0.011594889685511589          │
│   trk.double_majority_pt0.5_std_std    │                  nan                   │
│       trk.double_majority_pt0.9        │           0.8916953802108765           │
│     trk.double_majority_pt0.9_std      │          0.012939689680933952          │
│   trk.double_majority_pt0.9_std_std    │                  nan                   │
│       trk.double_majority_pt1.5        │           0.8886311054229736           │
│     trk.double_majority_pt1.5_std      │          0.025097912177443504          │
│   trk.double_majority_pt1.5_std_std    │                  nan                   │
│        trk.double_majority_std         │           0.0091920280829072           │
│      trk.double_majority_std_std       │                  nan                   │
│        trk.fake_double_majority        │           0.108233243227005            │
│     trk.fake_double_majority_pt0.5     │          0.053272224962711334          │
│   trk.fake_double_majority_pt0.5_std   │          

[{'attractive': 1.3140039300196804e-05,
  'repulsive': 2.991704059240874e-05,
  'background': 1.0,
  'attractive_weighted': 1.3140039300196804e-05,
  'repulsive_weighted': 2.223974297521636e-05,
  'background_weighted': 0.00409999955445528,
  'total': 0.004135379567742348,
  'v_measure': 0.9078373908996582,
  'homogeneity': 0.9336580634117126,
  'completeness': 0.8834384679794312,
  'trk.n_particles': 7648.39990234375,
  'trk.n_cleaned_clusters': 4767.60009765625,
  'trk.perfect': 0.2775444984436035,
  'trk.double_majority': 0.5153970718383789,
  'trk.lhc': 0.9037803411483765,
  'trk.fake_perfect': 0.3460858464241028,
  'trk.fake_double_majority': 0.108233243227005,
  'trk.fake_lhc': 0.09621964395046234,
  'trk.n_particles_pt0.5': 3414.800048828125,
  'trk.n_cleaned_clusters_pt0.5': 2834.0,
  'trk.perfect_pt0.5': 0.48133400082588196,
  'trk.double_majority_pt0.5': 0.7771095037460327,
  'trk.lhc_pt0.5': 0.9290783405303955,
  'trk.fake_perfect_pt0.5': 0.3490477204322815,
  'trk.fake_doub